<a href="https://colab.research.google.com/github/prabirdeb/Cardiovascular-Risk-Prediction/blob/main/Cardiovascular_Risk_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cardiovascular Risk Prediction**

# **Steps of Exploratory Data Analysis (EDA)**

**1. Data or EXPERIENCE**

Here, we are provided with a dataset of cardiovascular disease.

Thus, the dataset is actually a collection of experiences about patients with cardiovascular disease. 

Now, we need to **decode the set of experiences** to help in building a model for Cardiovascular Risk Prediction.

At first, we import the libraries or functions for **making our journey easy** and then **get connected** to the set of experiences.

In [1]:
#importing libraries
import numpy as np
import pandas as pd
from termcolor import colored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Reading the data as pandas dataframe
cardio_df=pd.read_csv('/content/drive/MyDrive/Almabetter Assignments/Capstone projects/Cardiovascular Risk Prediction-Prabir Debnath/data_cardiovascular_risk.csv')

**2. Features or "DHARMA":**

Here, the columns are the set of features, showing the way to reach final decoded experience or conclusions.  

As there is a huge no. of experiences, we cannot see the whole lot of experiences and therefore we find the features on the **data head**.

In [4]:
cardio_df.head(2)

,id,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,0,64,2.0,F,YES,3.0,0.0,0,0,0,221.0,148.0,85.0,NaN,90.0,80.0,1
1,1,36,4.0,M,NO,0.0,0.0,0,1,0,212.0,168.0,98.0,29.77,72.0,75.0,0


**3. Statistics or MEASUREMENTS**

As there is a huge no. of experiences, we take the help of statistics to **measure** each and every features in different dimensions and thus step by step, will find the most important features or the exact way to decode the experiences.

“**what gets measured gets done**“.

In [5]:
cardio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3390 entries, 0 to 3389
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               3390 non-null   int64  
 1   age              3390 non-null   int64  
 2   education        3303 non-null   float64
 3   sex              3390 non-null   object 
 4   is_smoking       3390 non-null   object 
 5   cigsPerDay       3368 non-null   float64
 6   BPMeds           3346 non-null   float64
 7   prevalentStroke  3390 non-null   int64  
 8   prevalentHyp     3390 non-null   int64  
 9   diabetes         3390 non-null   int64  
 10  totChol          3352 non-null   float64
 11  sysBP            3390 non-null   float64
 12  diaBP            3390 non-null   float64
 13  BMI              3376 non-null   float64
 14  heartRate        3389 non-null   float64
 15  glucose          3086 non-null   float64
 16  TenYearCHD       3390 non-null   int64  
dtypes: float64(9),

In [6]:
cardio_df.describe()

,id,age,education,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,3390.000000,3390.000000,3303.000000,3368.000000,3346.000000,3390.000000,3390.000000,3390.000000,3352.000000,3390.00000,3390.000000,3376.000000,3389.000000,3086.000000,3390.000000
mean,1694.500000,49.542183,1.970936,9.069477,0.029886,0.006490,0.315339,0.025664,237.074284,132.60118,82.883038,25.794964,75.977279,82.086520,0.150737
std,978.753033,8.592878,1.019081,11.879078,0.170299,0.080309,0.464719,0.158153,45.247430,22.29203,12.023581,4.115449,11.971868,24.244753,0.357846
min,0.000000,32.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,107.000000,83.50000,48.000000,15.960000,45.000000,40.000000,0.000000
25%,847.250000,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,206.000000,117.00000,74.500000,23.020000,68.000000,71.000000,0.000000
50%,1694.500000,49.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,234.000000,128.50000,82.000000,25.380000,75.000000,78.000000,0.000000
75%,2541.750000,56.000000,3.000000,20.000000,0.000000,0.000000,1.000000,0.000000,264.000000,144.00000,90.000000,28.040000,83.000000,87.000000,0.000000
max,3389.000000,70.000000,4.000000,70.000000,1.000000,1.000000,1.000000,1.000000,696.000000,295.00000,142.500000,56.800000,143.000000,394.000000,1.000000


In [7]:
for column_name in cardio_df.columns:
  print(colored(f"Unique values for {column_name}:",'blue', attrs=['bold']))
  print(cardio_df[column_name].unique())

Unique values for id:
[   0    1    2 ... 3387 3388 3389]
Unique values for age:
[64 36 46 50 61 41 55 53 43 44 58 51 56 42 38 63 35 54 62 47 34 39 59 45
 49 60 40 52 48 37 65 57 68 67 33 66 69 70 32]
Unique values for education:
[ 2.  4.  1.  3. nan]
Unique values for sex:
['F' 'M']
Unique values for is_smoking:
['YES' 'NO']
Unique values for cigsPerDay:
[ 3.  0. 10. 20. 30. 35. 40. 15.  9. 60. 17. 25.  5. 43.  7. 50. 19. 18.
  1. 45.  2.  6. nan 11.  8.  4. 14. 16. 23. 13. 12. 70. 38.]
Unique values for BPMeds:
[ 0. nan  1.]
Unique values for prevalentStroke:
[0 1]
Unique values for prevalentHyp:
[0 1]
Unique values for diabetes:
[0 1]
Unique values for totChol:
[221. 212. 250. 233. 241. 272. 238. 295. 220. 326.  nan 210. 213. 227.
 188. 240. 257. 267. 232. 199. 175. 164. 226. 248. 245. 279. 273. 252.
 198. 266. 216. 237. 193. 293. 180. 206. 215. 208. 262. 270. 292. 346.
 202. 340. 334. 351. 235. 243. 196. 275. 186. 190. 268. 234. 264. 156.
 260. 225. 183. 246. 174. 288. 254. 150. 21

In [8]:
for column_name in cardio_df.columns:
  print(colored(f"No. of unique values for {column_name}:",'green', attrs=['bold']))
  print(cardio_df[column_name].nunique())

No. of unique values for id:
3390
No. of unique values for age:
39
No. of unique values for education:
4
No. of unique values for sex:
2
No. of unique values for is_smoking:
2
No. of unique values for cigsPerDay:
32
No. of unique values for BPMeds:
2
No. of unique values for prevalentStroke:
2
No. of unique values for prevalentHyp:
2
No. of unique values for diabetes:
2
No. of unique values for totChol:
240
No. of unique values for sysBP:
226
No. of unique values for diaBP:
142
No. of unique values for BMI:
1259
No. of unique values for heartRate:
68
No. of unique values for glucose:
132
No. of unique values for TenYearCHD:
2


In [9]:
for column_name in cardio_df.columns:
  print(colored(f"No. of null values for {column_name}:",'magenta', attrs=['bold']))
  print(cardio_df[column_name].isnull().sum())

No. of null values for id:
0
No. of null values for age:
0
No. of null values for education:
87
No. of null values for sex:
0
No. of null values for is_smoking:
0
No. of null values for cigsPerDay:
22
No. of null values for BPMeds:
44
No. of null values for prevalentStroke:
0
No. of null values for prevalentHyp:
0
No. of null values for diabetes:
0
No. of null values for totChol:
38
No. of null values for sysBP:
0
No. of null values for diaBP:
0
No. of null values for BMI:
14
No. of null values for heartRate:
1
No. of null values for glucose:
304
No. of null values for TenYearCHD:
0


In [ ]:
# Finding out the final important columns considering type of unique values, less no. of unique values and less no. of null values
# No columns to remove

**4. Data Cleaning**

Now we can create a clean experience set with important features. 

In this step, we treat the missing values or "?" values through imputation if required.

If there is any string containing a Python literal in any of the important feature, we must evaluate them

We may also create new feature if found important during the analysis.

We check the statistics further on clean data.

In [10]:
for column_name in cardio_df.columns:
  print(colored(f"Value counts for {column_name}:", attrs=['bold']))
  print(cardio_df[column_name].value_counts())

Value counts for id:
2047    1
2576    1
2624    1
573     1
2620    1
       ..
1122    1
3167    1
1118    1
3163    1
0       1
Name: id, Length: 3390, dtype: int64
Value counts for age:
40    148
42    145
41    144
46    140
39    139
44    135
48    134
45    131
43    127
38    119
52    118
55    116
50    115
51    114
53    114
47    108
54    107
49    104
57     97
60     95
58     95
59     93
63     93
56     92
61     87
62     80
36     75
64     75
37     73
65     43
67     33
66     30
35     29
34     16
68     14
69      5
33      4
70      2
32      1
Name: age, dtype: int64
Value counts for education:
1.0    1391
2.0     990
3.0     549
4.0     373
Name: education, dtype: int64
Value counts for sex:
F    1923
M    1467
Name: sex, dtype: int64
Value counts for is_smoking:
NO     1703
YES    1687
Name: is_smoking, dtype: int64
Value counts for cigsPerDay:
0.0     1703
20.0     606
30.0     176
15.0     172
10.0     106
9.0      104
5.0      103
3.0       79
40.0   

**5. Data Visualization**

When we know all the important features of our experiences, we can go a step ahead by finding the relationship among features. Here, we take the help of visualization because

**"A picture is worth a thousand words"**

**6. Anomaly Detection**

While, we are finding out the **general formula** from the experiences, we should identify the outlier or **exceptional observations** for all the important features and keep them aside during the analysis.

**7. Conclusion**